# Requests

## Beloved Features  功能特性

- Keep-Alive & Connection Pooling
- Internatioal Domains and URLs
- Sessions with Cookie Persistence 带持久Cookie的会话
- Browser-style SSL Verification 浏览器式的SSL认证
- Automatic Content Decoding
- Basic/Digest Authentication 基本/摘要式的身份认证
- Elegant Key/Value Cookies
- Automatic Decompression 自动解压
- Unicode Response Bodies
- HTTP(S) Proxy Support
- Multipart File Uploads 文件分块上传
- Streaming Downloads
- Connection Timeouts
- Chunked Requests 分块请求
- `.netrc` Support 

## Quickstart

### Make a Request

In [1]:
import requests

r = requests.get('https://api.github.com/events')
r = requests.post('https://httpbin.org/post', data={'key': 'value'})
r = requests.put('https://httpbin.org/put', data={'key': 'value'})
r = requests.delete('https://httpbin.org/delete')
r = requests.get('https://httpbin.org/get')
r = requests.options('https://httpbin.org/get')

### Passing Parameters In URLs

In [2]:
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.get('https://httpbin.org/get', params=payload)
print(r.url)

https://httpbin.org/get?key1=value1&key2=value2


In [3]:
payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
r = requests.get('https://httpbin.org/get', params=payload)
print(r.url)

https://httpbin.org/get?key1=value1&key2=value2&key2=value3


### Response Content

In [4]:
r = requests.get('https://api.github.com/events')
r.text[:200]

'[{"id":"9413450481","type":"PullRequestEvent","actor":{"id":7237365,"login":"chenjiahan","display_login":"chenjiahan","gravatar_id":"","url":"https://api.github.com/users/chenjiahan","avatar_url":"htt'

In [5]:
r.encoding

'utf-8'

In [6]:
r.encoding = 'ISO-8859-1'

### Binary Response Content

In [7]:
r.content[:200]

b'[{"id":"9413450481","type":"PullRequestEvent","actor":{"id":7237365,"login":"chenjiahan","display_login":"chenjiahan","gravatar_id":"","url":"https://api.github.com/users/chenjiahan","avatar_url":"htt'

**The `gzip` and `deflate` transfer-encodings are automatically decoded for you.**

In [8]:
from PIL import Image
from io import BytesIO

r = requests.get('https://www.baidu.com/img/xinshouye_46cc6be3783724af1729ba51cfcde494.png')
i = Image.open(BytesIO(r.content))
i.save('baidu.png')

### Json Response Content

In [9]:
r = requests.get('https://api.github.com/events')
r.json()[0]

{'id': '9413452475',
 'type': 'WatchEvent',
 'actor': {'id': 33881637,
  'login': 'zhenzhen2017',
  'display_login': 'zhenzhen2017',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/zhenzhen2017',
  'avatar_url': 'https://avatars.githubusercontent.com/u/33881637?'},
 'repo': {'id': 65869182,
  'name': 'foolwood/benchmark_results',
  'url': 'https://api.github.com/repos/foolwood/benchmark_results'},
 'payload': {'action': 'started'},
 'public': True,
 'created_at': '2019-04-10T07:50:48Z'}

**It should be noted that the success of the call to `r.json()` does not indicate the success of the response.Some servers may return a JSON object in a failed response (e.g. error details with HTTP 500). Such JSON will be decoded and returned. To check that a request is successful, use `r.raise_for_status()` or check `r.status_code` is what you expect.**

In [10]:
r.status_code

200

In [11]:
r = requests.get('https://httpbin.org/status/200')
print(r.raise_for_status() is None)

True


In [12]:
bad_r = requests.get('https://httpbin.org/status/404')
bad_r.raise_for_status()

HTTPError: 404 Client Error: NOT FOUND for url: https://httpbin.org/status/404

### Raw Response Content

**get the raw socket response from the server**

In [13]:
r = requests.get('https://api.github.com/events', stream=True)
r.raw

In [14]:
r.raw.read(200)

b"\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x03\xed}\xf9\x93\x1cW\x9d\xe7\xbf\x92\xdb\x10\x81\x1d\xb8\xaa\xf2>:\x8c\x91l\x1d\x88U\xb7l\xa9}` \x9a<^v\xa7\xba\xba\xb2\\Y\xa5\xbe\xc6\x11b\xb0\x19vm\xae\x05\xc60\xc3\xc2@\x8cw\x196Vbc\x07\xc2\xc68\xf8[\x1c\xab\x96\xac\x9fv\xff\x84\xfd\xbc\xbc3_VU\x1e%\x96\x98-G\xd8\xee\xca\xcc\xf7y\xf7\xf7}\xdf\xf7\xfc\xea\xd9\x86\xe7lln\x18\xb2 \xc9\x8a,\x1a\xfa\xc63\x1b\xd3\x931\xc1\xb3\x17g\xc1\xfe\xe5;d4\xc5#\xd3\x9e\xfa\x93\x8d\xcd\xe8kE3$UU\x9e\xd9\x18\xfa{\xde\x08_NL\xdb\x9cLx\x01\x1f:^0\x1e\x9a'\xbb\x15\xaf\xf6&\xe6\x1dsjNv\xc3\x1a\xf1\xedl2\xc4\x1f"

**In general, however, you should use a pattern like this to save what is being streamed to a file:**

In [16]:
filename = 'api_github_events.txt'
with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

***Note:*** An important note about using `Response.iter_content` versus `Response.raw`. `Response.iter_content` will automatically decode the `gzip` and `deflate` transfer-encodings. `Response.raw` is a raw stream of bytes – it does not transform the response content. If you really need access to the bytes as they were returned, use `Response.raw`.

### Custom Headers

In [17]:
url = 'https://api.github.com/some/endpoint'
headers = {'user-agent': 'my-app/0.0.1'}
r = requests.get(url, headers=headers)

***Note:*** **Custom headers are given less precedence than more specific sources of information. For instance:**
- Authorization headers set with headers= will be overridden if credentials are specified in `.netrc`, which in turn will be overridden by the `auth=` parameter. (headers < `.netrc` < `auth`)
- Authorization headers will be removed if you get redirected off-host.
- Proxy-Authorization headers will be overridden by proxy credentials provided in the URL.
- Content-Length headers will be overridden when we can determine the length of the content.

### More complicated POST requests

In [18]:
payload = {'key1': 'value1', 'key2': 'value2'}

r = requests.post('https://httpbin.org/post', data=payload)
print(r.text[:200])

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content


In [19]:
payload_tuples = [('key1', 'value1'), ('key1', 'value2')]
r1 = requests.post('https://httpbin.org/post', data=payload_tuples)
payload_dict = {'key1': ['value1', 'value2']}
r2 = requests.post('https://httpbin.org/post', data=payload_dict)
print(r1.text[:200])
print(r1.text == r2.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": [
      "value1", 
      "value2"
    ]
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    
True


**There are times that you may want to send data that is `not form-encoded(Content-Type=application/json)`. If you pass in a `string` instead of a `dict`, that data will be posted directly.**

For example, the GitHub API v3 accepts JSON-Encoded POST/PATCH data:

In [20]:
import json

url = 'https://api.github.com/some/endpoint'
payload = {'some': 'data'}

r = requests.post(url, data=json.dumps(payload))

**Instead of encoding the dict yourself, you can also pass it directly using the json parameter (added in version 2.4.2) and it will be encoded automatically:**

In [21]:
payload = {'some': 'data'}

r = requests.post(url, json=payload)

***Note,*** **the json parameter is ignored if either data or files is passed.**

**Using the json parameter in the request will change the Content-Type in the header to application/json.**

### POST a Multipart-Encoded File

In [22]:
url = 'https://httpbin.org/post'
files = {'file': open('baidu.png', 'rb')}

r = requests.post(url, files=files)
r.text[:200]

'{\n  "args": {}, \n  "data": "", \n  "files": {\n    "file": "data:application/octet-stream;base64,iVBORw0KGgoAAAANSUhEUgAAAhwAAAECCAMAAACCFP44AAADAFBMVEUAAAD9/fv//////////////////////////////////////////'

**You can set the filename, content_type and headers explicitly.**

**you can send strings to be received as files:**

In [23]:
url = 'https://httpbin.org/post'
files = {'file': ('report.csv', 'some,data,to,send\nanother,row,to,send\n')}

r = requests.post(url, files=files)
r.text[:200]

'{\n  "args": {}, \n  "data": "", \n  "files": {\n    "file": "some,data,to,send\\nanother,row,to,send\\n"\n  }, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n '

**In the event you are posting a very large file as a `multipart/form-data` request, you may want to stream the request. By default, `requests` does not support this, but there is a separate package which does - `requests-toolbelt`. You should read [the toolbelt’s documentation](https://toolbelt.readthedocs.io/en/latest/) for more details about how to use it.**

**For sending multiple files in one request refer to the [advanced](http://docs.python-requests.org/en/master/user/advanced/#advanced) section.**

***Warning:*** It is strongly recommended that you open files in binary mode. This is because Requests may attempt to provide the Content-Length header for you, and if it does this value will be set to the number of bytes in the file. Errors may occur if you open the file in text mode.

### Response Status Codes

In [24]:
r = requests.get('https://httpbin.org/get')
r.status_code

200

**Requests also comes with a built-in status code lookup object for easy reference:**

In [25]:
r.status_code == requests.codes.ok

True

In [26]:
r.raise_for_status() == None

True

In [27]:
bad_r = requests.get('https://httpbin.org/status/404')
bad_r.status_code

404

In [28]:
bad_r.raise_for_status()

HTTPError: 404 Client Error: NOT FOUND for url: https://httpbin.org/status/404

### Response Headers

In [29]:
r.headers

{'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json', 'Date': 'Wed, 10 Apr 2019 07:57:43 GMT', 'Server': 'nginx', 'Content-Length': '183', 'Connection': 'keep-alive'}

**The dictionary is special, though: it’s made just for HTTP headers. According to [RFC 7230](https://tools.ietf.org/html/rfc7230#section-3.2), HTTP Header names are case-insensitive.**

In [30]:
r.headers['content-type'], r.headers['CONTENT-TYPE']

('application/json', 'application/json')

In [31]:
headers = r.headers
headers['content-type'], headers['CONTENT-TYPE']

('application/json', 'application/json')

**It is also special in that the server could have sent the same header multiple times with different values, but requests combines them so they can be represented in the dictionary within a single mapping, as per [RFC 7230](https://tools.ietf.org/html/rfc7230#section-3.2):**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A recipient MAY combine multiple header fields with the same field name into one “field-name: field-value” pair, without changing the semantics of the message, by appending each subsequent field value to the combined field value in order, separated by a comma.

### Cookies

In [32]:
url = 'http://httpbin.org/cookies'
cookies = dict(cookies_are='working')

r = requests.get(url, cookies=cookies)
r.text

'{\n  "cookies": {\n    "cookies_are": "working"\n  }\n}\n'

In [33]:
r.cookies.get_dict()

{}

In [34]:
type(r.cookies)

requests.cookies.RequestsCookieJar

**Cookies are returned in a [RequestsCookieJar](http://docs.python-requests.org/zh_CN/latest/api.html#requests.cookies.RequestsCookieJar), which acts like a dict but also offers a more complete interface, suitable for use over multiple domains or paths.**
<br>Cookie 的返回对象为 RequestsCookieJar，它的行为和字典类似，但接口更为完整，适合跨域名跨路径使用。

In [35]:
jar = requests.cookies.RequestsCookieJar()
jar.set('tasty_cookie', 'yum', domain='httpbin.org', path='/cookies')
jar.set('gross_cookie', 'blech', domain='httpbin.org', path='/elsewhere')
url = 'https://httpbin.org/cookies'
r = requests.get(url, cookies=jar)
r.text

'{\n  "cookies": {\n    "tasty_cookie": "yum"\n  }\n}\n'

### Redirection and History

**By default Requests will perform location redirection for all verbs except HEAD.**

**The [Response.history](http://docs.python-requests.org/en/master/api/#requests.Response.history) list contains the [Response](http://docs.python-requests.org/en/master/api/#requests.Response) objects that were created in order to complete the request. The list is sorted from the oldest to the most recent response.**

In [36]:
r = requests.get('http://github.com/')
r.url

'https://github.com/'

In [37]:
r.status_code

200

In [38]:
r.history

[<Response [301]>]

**If you’re using GET, OPTIONS, POST, PUT, PATCH or DELETE, you can disable redirection handling with the `allow_redirects` parameter:**

In [39]:
r = requests.get('http://github.com/', allow_redirects=False)
r.status_code, r.history

(301, [])

**If you’re using HEAD, you can enable redirection as well:**

In [40]:
r = requests.head('http://github.com/', allow_redirects=True)
r.status_code, r.history

(200, [<Response [301]>])

### Timeouts

In [41]:
r = requests.get('https://github.com/', timeout=0.5)
# ReadTimeout, ConnectTimeout

ReadTimeout: HTTPSConnectionPool(host='github.com', port=443): Read timed out. (read timeout=0.5)

***Note:*** timeout is not a time limit on the entire response download; rather, an exception is raised if the server has not issued a response for timeout seconds (more precisely, if no bytes have been received on the underlying socket for timeout seconds). If no timeout is specified explicitly, requests do not time out.
<br>timeout 仅对连接过程有效，与响应体的下载无关。

### Errors and Exceptions

- In the event of a network problem (e.g. DNS failure, refused connection, etc), Requests will raise a **`ConnectionError`** exception.
- [**Response.raise_for_status()**](http://docs.python-requests.org/en/master/api/#requests.Response.raise_for_status) will raise an **`HTTPError`** if the HTTP request returned an unsuccessful status code.
- If a request times out, a **`Timeout`** exception is raised.
- If a request exceeds the configured number of maximum redirections, a **`TooManyRedirects`** exception is raised.
- All exceptions that Requests explicitly raises inherit from **`requests.exceptions.RequestException`**.